In [1]:
!pip install ultralytics
!pip install split-folders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 6.8 MB/s eta 0:00:00


In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunny7712","key":"f59af526f9ec5fc5f75e10ffa0cb9efb"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rakibuleceruet/drowsiness-prediction-dataset
! unzip drowsiness-prediction-dataset.zip

In [5]:
import os
import splitfolders

splitfolders.ratio('0 FaceImages', output="splitted_data", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 9120 files [00:10, 852.91 files/s]


In [6]:
from ultralytics import YOLO

In [9]:
model =  YOLO('yolov8n-cls.pt')
model.train(data = '/content/splitted_data', epochs = 3, imgsz = 224)

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/splitted_data, epochs=3, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, op

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7afccce663e0>
curves: []
curves_results: []
fitness: 0.9753289520740509
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9506579041481018, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9753289520740509}
save_dir: PosixPath('runs/classify/train3')
speed: {'preprocess': 0.06897407665587307, 'inference': 0.519972098501105, 'loss': 0.0005811452865600586, 'postprocess': 0.0005136979253668534}
task: 'classify'
top1: 0.9506579041481018
top5: 1.0

In [ ]:
loaded = YOLO('/content/runs/classify/train3/weights/best.pt')

y_actual = []
preds = []



# r = loaded('/content/splitted_data/test/Fatigue Subjects/0.jpg')
# preds.append(r[0].probs.top1)
# print(r[0].probs.top1)
# y_actual.append(1)

for name in os.listdir('/content/splitted_data/test/Active Subjects'):
    try:
        r = loaded(os.path.join('/content/splitted_data/test/Active Subjects', name))
        preds.append(r[0].probs.top1)
        # print(r[0].probs.top1)
        y_actual.append(0)
    except:
        continue

for name in os.listdir('/content/splitted_data/test/Fatigue Subjects'):
    try:
        r = loaded(os.path.join('/content/splitted_data/test/Fatigue Subjects', name))
        preds.append(r[0].probs.top1)
        y_actual.append(1)
    except:
        continue

In [52]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

print(f'Acc : {accuracy_score(y_actual, preds)}')
print(f'f1 : {f1_score(y_actual, preds)}')
print(f'roc_auc : {roc_auc_score(y_actual, preds)}')

Acc : 0.9539473684210527
f1 : 0.9535398230088495
roc_auc : 0.9539473684210527
